In [ ]:
from fbCrawler.models import (Product, Members, Orders, Helper, Session, Group, Checkout, BankStatement,settlementRule)
from fbCrawler.utils import *
import pandas as pd
import os, json, datetime
from django.contrib.auth.models import User
from django.utils import timezone
from django.conf import settings
from django.utils.timezone import make_aware
from django.db.models import Q
import io
import csv

# 熱門度 View

In [12]:
session_id = '20190911'


# 物件關聯,兩種寫法
# session = Session.objects.get(session_id=session_id)
# ods = Orders.objects.filter(session=session).filter(~Q(amount=0))
currentSession = Session.objects.get(session_id='20190911')
ods = Orders.objects.filter(session__session_id=session_id).filter(~Q(amount=0))

# list(ods.values()),objects轉pandas
df_ods = pd.DataFrame(
    list(ods.values('member_id', 'member__name',
                    'product__wine_id', 'product__name', 'amount','product__product_article_link',
                    'product__price', 'product__erp_no')
         )
)
df_ods['order_total_price'] = df_ods['amount'] * \
                              df_ods['product__price']

#提取品項規則
sr = settlementRule.objects.filter(session=session_id)
df_sr = pd.DataFrame(
    list(sr.values('wineID', 'wineName', 'price', 'stockNum', 'stockNumFalse', 'limit', 'sale', 'deliverFree')))
df_sr = df_sr.set_index('wineID')

df_ods_group = df_ods.groupby(['product__wine_id','product__product_article_link'])['amount', 'order_total_price'].sum()
df_ods_group['bottleCount']  = df_ods.groupby(['product__wine_id','product__product_article_link'])['amount'].count()
df_ods_group=df_ods_group.reset_index().set_index('product__wine_id')
df = pd.concat([df_ods_group, df_sr], axis=1)
df['DSR'] = round((df['amount'].astype(int)) / (df['stockNumFalse'].astype(int)) * 100)  # 供需比
df['limit'] = df['limit'].apply(lambda s: str(s).replace('0', '無'))
df['bgc'] = ['#fccf80' if i > 100 else 'white' for i in df['DSR']]
df = df.sort_values('DSR', ascending=False)
df = df.reset_index()
df = df.rename(columns={'index': 'wine_id'})
df2 = df.T.to_dict().values()
context = {
    'session': currentSession,
    'datas': list(df2)
}
context


{'datas': [{'wine_id': '4520',
   'product__product_article_link': 'https://www.facebook.com/groups/1667261116828377/permalink/2376136245940857/',
   'amount': 148,
   'order_total_price': 263440,
   'bottleCount': 114,
   'deliverFree': '0',
   'limit': '1',
   'price': '1780',
   'sale': '0',
   'stockNum': '25',
   'stockNumFalse': '15',
   'wineName': 'Ayala 2008 Rosé No.8 Brut Champagne',
   'DSR': 987.0,
   'bgc': '#fccf80'},
  {'wine_id': '4521',
   'product__product_article_link': 'https://www.facebook.com/groups/1667261116828377/permalink/2376139212607227/',
   'amount': 159,
   'order_total_price': 108120,
   'bottleCount': 114,
   'deliverFree': '0',
   'limit': '無',
   'price': '680',
   'sale': '0',
   'stockNum': '31',
   'stockNumFalse': '19',
   'wineName': 'Albert Bichot 2015 Bourgogne Vieilles Vignes紅',
   'DSR': 837.0,
   'bgc': '#fccf80'},
  {'wine_id': '4490',
   'product__product_article_link': 'https://www.facebook.com/groups/1667261116828377/permalink/2376138525

# 銷售列表

In [10]:
currentSession = Session.objects.get(session_id='20190911')
ods = Orders.objects.filter(session=currentSession).filter(~Q(amount=0))
# ods = Orders.objects.filter(session__session_id=session_id).filter(~Q(amount=0))

# list(ods.values()),objects轉pandas
df_ods = pd.DataFrame(
    list(ods.values('member_id', 'member__name',
                    'product__wine_id', 'product__name', 'product__product_article_link', 'amount',
                    'product__price', 'product__erp_no')
         )
)
df_ods['order_total_price'] = df_ods['amount'] * \
                              df_ods['product__price']

sr = settlementRule.objects.filter(session=currentSession)
df_sr = pd.DataFrame(
    list(sr.values('wineID', 'wineName', 'price', 'stockNum', 'stockNumFalse', 'limit', 'sale', 'deliverFree')))
df_sr = df_sr.set_index('wineID')

df_ods_group = df_ods.groupby(['product__wine_id', 'product__product_article_link'])[
    'amount', 'order_total_price'].sum()
df_ods_group['bottleCount'] = df_ods.groupby(['product__wine_id', 'product__product_article_link'])[
    'amount'].count()
df_ods_group = df_ods_group.reset_index().set_index('product__wine_id')

df = pd.concat([df_ods_group, df_sr], axis=1)
df['DSR'] = round((df['amount'].astype(int)) / (df['stockNum'].astype(int)) * 100)  # 供需比
df['bgc'] = ['#fccf80' if i >= 100 else 'white' for i in df['DSR']]
df['limit'] = df['limit'].apply(lambda s: str(s).replace('0', '無'))
df['amount'] = df['amount'].astype(int)
df['stockNum'] = df['stockNum'].astype(int)
df['price'] = df['price'].astype(int)
df['saleVal'] = [s if v > s else v for v, s in zip(df['amount'], df['stockNum'])]
df['settle_total_price'] = df['saleVal'] * df['price']
df = df.sort_values('DSR', ascending=False)
df = df.reset_index()
df = df.rename(columns={'index': 'wine_id'})
df2 = df.T.to_dict().values()
context = {
    'session': currentSession,
    'datas': list(df2),
    'total_income': df['settle_total_price'].sum()
}



In [11]:
context

{'session': <Session: 20190911>,
 'datas': [{'wine_id': '4520',
   'product__product_article_link': 'https://www.facebook.com/groups/1667261116828377/permalink/2376136245940857/',
   'amount': 148,
   'order_total_price': 263440,
   'bottleCount': 114,
   'deliverFree': '0',
   'limit': '1',
   'price': 1780,
   'sale': '0',
   'stockNum': 25,
   'stockNumFalse': '15',
   'wineName': 'Ayala 2008 Rosé No.8 Brut Champagne',
   'DSR': 592.0,
   'bgc': '#fccf80',
   'saleVal': 25,
   'settle_total_price': 44500},
  {'wine_id': '4521',
   'product__product_article_link': 'https://www.facebook.com/groups/1667261116828377/permalink/2376139212607227/',
   'amount': 159,
   'order_total_price': 108120,
   'bottleCount': 114,
   'deliverFree': '0',
   'limit': '無',
   'price': 680,
   'sale': '0',
   'stockNum': 31,
   'stockNumFalse': '19',
   'wineName': 'Albert Bichot 2015 Bourgogne Vieilles Vignes紅',
   'DSR': 513.0,
   'bgc': '#fccf80',
   'saleVal': 31,
   'settle_total_price': 21080},
  {

In [ ]:
# currentSession = Session.objects.get(session_id='20190911')
# sessionProducts = Product.objects.filter(session=currentSession)
# productList = list(sessionProducts)

# total_income = 0
# for product in productList:
#     productTotalOrders = Orders.objects.filter(
#         product=product).aggregate(Sum('amount'))['amount__sum']
#     product.totalOrders = productTotalOrders

#     productIncome = productTotalOrders*product.price
#     product.income = productIncome
#     total_income += productIncome

# context = {
#     'session': currentSession,
#     'products': productList,
#     'total_income': total_income
# }

In [ ]:
context = {
    'message': None,
    'sessionExist': False,
    'hasOrder': False,
    'orders': None,
    'session': None,
    'member': None,
    'total_price': 0
}
# Session id 不存在
session_qs = Session.objects.filter(session_id=session_id)

if session_qs.count() == 0:
    context['message'] = '團次錯誤，請到團次列表查詢參與團次'
    return render(request, 'session/updateOrder.html', context=context)

session = session_qs[0]
if session.status != 'update_orders':
    context['message'] = '本團次目前沒有開放修正數量，有疑問請洽小幫手'
    return render(request, 'session/updateOrder.html', context=context)
else:
    context['sessionExist'] = True

session = session_qs[0]
member = Members.objects.get(user=request.user)
# 查詢此用戶在此 session 有無訂單
orders_qs = Orders.objects.filter(member=member, session=session)


if orders_qs.count() < 1:
    context['message'] = '您並無本團次訂單，有疑問請洽小幫手'
    return render(request, 'session/updateOrder.html', context=context)
else:
    context['hasOrder'] = True
    context['orders'] = orders_qs
    context['session'] = session
    context['member'] = member
    # total price
    total_price = 0
    for order in orders_qs:
        total_price += order.amount*order.product.price
    context['total_price'] = total_price

# 購物車資訊

In [2]:
context = {
    'message': None,
    'orders': None,
    'session': None,
    'member': None,
    'total_price': 0
}


#物件關聯,兩種寫法
session = Session.objects.get(session_id='20190911')
orders_qs  = Orders.objects.filter(session=session)
# ods = Orders.objects.filter(session__session_id=session_id).filter(~Q(amount=0))

#list(ods.values()),objects轉pandas
df_ods = pd.DataFrame(
    list(orders_qs .values('member_id', 'member__name', 
            'product__wine_id', 'product__name','product__product_article_link','amount',
            'product__price', 'product__erp_no')
    )
)

orders_qs2  = Orders.objects.filter(session=session)
df_ods2 = pd.DataFrame(list(orders_qs2 .values('member_id','product__wine_id','amount')))
df_ods2=df_ods2.set_index('product__wine_id')
df_ods2['total_amount']=df_ods2.groupby('product__wine_id')['amount'].sum()


df_ods=df_ods.set_index('product__wine_id')
sr = settlementRule.objects.filter(session=session)
df_sr=pd.DataFrame(list(sr.values('wineID','stockNum','stockNumFalse','limit','sale','deliverFree')))
df_sr=df_sr.set_index('wineID')
for i in ['stockNum','stockNumFalse','limit','sale','deliverFree']:
    df_ods[i]=df_sr[i]

df_ods['total_amount']=df_ods2['total_amount']
df_ods['amount']=df_ods['amount'].astype(int)
df_ods['stockNumFalse']=df_ods['stockNumFalse'].astype(int)
df_ods['bgc']=df_ods['total_amount']-df_ods['stockNumFalse']
df_ods['limit']=df_ods['limit'].apply(lambda s:str(s).replace('0','無'))
df=df_ods.sort_values('bgc',ascending=True)
df=df.reset_index()
orders_qs=df.T.to_dict().values()

context['orders'] = list(orders_qs)
total_price = 0
for order in orders_qs:
    total_price += order['amount']*order['product__price']
context['total_price'] = total_price
df

,product__wine_id,amount,member__name,member_id,product__erp_no,product__name,product__price,product__product_article_link,stockNum,stockNumFalse,limit,sale,deliverFree,total_amount,bgc
0,4298,1,余虹達,135,101004298,Mauro Molino 2018 Roero Arneis DOCG 干白酒,580,https://www.facebook.com/groups/16672611168283...,1200,720,無,0,0,119,-601
1,4298,2,Wei Chung Su,1401,101004298,Mauro Molino 2018 Roero Arneis DOCG 干白酒,580,https://www.facebook.com/groups/16672611168283...,1200,720,無,0,0,119,-601
2,4298,1,李漢銘,2455,101004298,Mauro Molino 2018 Roero Arneis DOCG 干白酒,580,https://www.facebook.com/groups/16672611168283...,1200,720,無,0,0,119,-601
3,4298,2,洪一弘,1062,101004298,Mauro Molino 2018 Roero Arneis DOCG 干白酒,580,https://www.facebook.com/groups/16672611168283...,1200,720,無,0,0,119,-601
4,4298,1,范祥偉,371,101004298,Mauro Molino 2018 Roero Arneis DOCG 干白酒,580,https://www.facebook.com/groups/16672611168283...,1200,720,無,0,0,119,-601
5,4298,2,James Yin,52,101004298,Mauro Molino 2018 Roero Arneis DOCG 干白酒,580,https://www.facebook.com/groups/16672611168283...,1200,720,無,0,0,119,-601
6,4298,1,Chun Tai Tsai,680,101004298,Mauro Molino 2018 Roero Arneis DOCG 干白酒,580,https://www.facebook.com/groups/16672611168283...,1200,720,無,0,0,119,-601
7,4298,1,Catherine Chang,910,101004298,Mauro Molino 2018 Roero Arneis DOCG 干白酒,580,https://www.facebook.com/groups/16672611168283...,1200,720,無,0,0,119,-601
8,4298,2,Joanne Liang,63,101004298,Mauro Molino 2018 Roero Arneis DOCG 干白酒,580,https://www.facebook.com/groups/16672611168283...,1200,720,無,0,0,119,-601
9,4298,1,Tsz Yan,5752,101004298,Mauro Molino 2018 Roero Arneis DOCG 干白酒,580,https://www.facebook.com/groups/16672611168283...,1200,720,無,0,0,119,-601


In [6]:
orders_qs2  = Orders.objects.filter(session=session)
df_ods2 = pd.DataFrame(list(orders_qs2 .values('member_id','product__wine_id','amount')))
df_ods2=df_ods2.set_index('product__wine_id')
df_ods2['total_amount']=df_ods2.groupby('product__wine_id')['amount'].sum()
total_amount=df_ods2['total_amount']
total_amount=total_amount[~total_amount.duplicated()]
total_amount

product__wine_id
4490     59
4644     22
4479     24
4323     37
4525    163
4535     32
4521    159
4496     57
4643     21
4520    148
4534     52
4540     41
4412     33
4417      1
4512     18
4473    209
4526     62
4530     28
4537    111
4523     60
4513     36
4538     38
4528     20
4692     13
4645      7
4646      8
4647     14
4694     19
4871     34
4668      6
4669      3
4609     10
4611     12
4506     68
4659      5
4301     23
4505     42
4465      2
4667     31
4302     11
4504     48
4662     16
4298    119
4503     55
4502     17
Name: total_amount, dtype: int64

In [9]:
df_ods2['total_amount']

product__wine_id
4490    59
4490    59
4490    59
4490    59
4490    59
4490    59
4490    59
4490    59
4490    59
4490    59
4490    59
4490    59
4490    59
4490    59
4490    59
4490    59
4490    59
4490    59
4490    59
4490    59
4490    59
4490    59
4490    59
4490    59
4490    59
4490    59
4490    59
4490    59
4490    59
4490    59
        ..
4502    17
4502    17
4502    17
4502    17
4502    17
4502    17
4502    17
4502    17
4502    17
4502    17
4502    17
4502    17
4502    17
4502    17
4502    17
4502    17
4502    17
4502    17
4502    17
4502    17
4502    17
4502    17
4502    17
4502    17
4502    17
4502    17
4502    17
4502    17
4502    17
4502    17
Name: total_amount, Length: 2524, dtype: int64

In [ ]:
#物件關聯,兩種寫法
session = Session.objects.get(session_id='20190911')

orders_qs2  = Orders.objects.filter(session=session)
df_ods2 = pd.DataFrame(list(orders_qs2 .values('member_id','product__wine_id','amount')))
df_ods2=df_ods2.set_index('product__wine_id')
df_ods2['total_amount']=df_ods2.groupby('product__wine_id')['amount'].sum()



df_ods
# df_ods['amount']=df_ods['amount'].astype(int)
# df_ods['stockNumFalse']=df_ods['stockNumFalse'].astype(int)
# df_ods['bgc']=df_ods['total_amount']-df_ods['stockNumFalse']
# df_ods['limit']=df_ods['limit'].apply(lambda s:str(s).replace('0','無'))
# df=df_ods.sort_values('bgc',ascending=True)
# df=df.reset_index()
# orders_qs=df.T.to_dict().values()


# context = {
#     'message': None,
#     'orders': None,
#     'session': None,
#     'member': None,
#     'total_price': 0
# }

# context['orders'] = list(orders_qs)
# total_price = 0
# for order in orders_qs:
#     total_price += order['amount']*order['product__price']
# context['total_price'] = total_price
df_ods

In [ ]:
order['amount']